<img src="./images/output-0.png" />

<img src="./images/output-1.png" />

<img src="./images/output-2.png" />

<img src="./images/output-4.png" />

<img src="./images/output-5.png" />

<img src="./images/output-6.png" />

<img src="./images/output-7.png" />

<img src="./images/output-8.png" />

<img src="./images/output-9.png" />

<img src="./images/output-10.png" />

<img src="./images/output-11.png" />

<img src="./images/output-12.png" />

<img src="./images/output-13.png" />

In [1]:
# Example One, getting frames off a webcam.
# basic_webcam.py
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    cv2.imshow('Basic Web Cam',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
# Example two, advance camera properties. 
import numpy as np
import cv2
import time

pass 
# create a map to keep track of all these names
prop_map = {
    "pos_msec":cv2.cv.CV_CAP_PROP_POS_MSEC,
    "pos_frame":cv2.cv.CV_CAP_PROP_POS_FRAMES,
    "avi_ratio":cv2.cv.CV_CAP_PROP_POS_AVI_RATIO ,
    "width":cv2.cv.CV_CAP_PROP_FRAME_WIDTH ,
    "height":cv2.cv.CV_CAP_PROP_FRAME_HEIGHT ,
    "fps":cv2.cv.CV_CAP_PROP_FPS ,
    "fourcc":cv2.cv.CV_CAP_PROP_FOURCC ,
    "frame_count":cv2.cv.CV_CAP_PROP_FRAME_COUNT,
    "format":cv2.cv.CV_CAP_PROP_FORMAT ,
    "mode":cv2.cv.CV_CAP_PROP_MODE ,
    "brightness":cv2.cv.CV_CAP_PROP_BRIGHTNESS ,
    "contrast":cv2.cv.CV_CAP_PROP_CONTRAST ,
    "saturation":cv2.cv.CV_CAP_PROP_SATURATION,
    "hue":cv2.cv.CV_CAP_PROP_HUE ,
    "gain":cv2.cv.CV_CAP_PROP_GAIN ,
    "exposure":cv2.cv.CV_CAP_PROP_EXPOSURE ,
    "convert_rgb":cv2.cv.CV_CAP_PROP_CONVERT_RGB ,
 #   "white_balance":cv2.cv.CV_CAP_PROP_WHITE_BALANCE ,
    "rectification":cv2.cv.CV_CAP_PROP_RECTIFICATION}

# get a camera property
def get_prop(cam,name,prop_map):
    return cam.get(prop_map[name])

# set a camera property
def set_prop(cam,name,prop_map,value):
    cam.set(prop_map[name],value)

# print out all of the properites
def poll_props(cam,prop_map):
    out_map = {}
    for k,v in prop_map.items():
        result = cam.get(v)
        if( result == -1.0 ):
            out_map[k] = None
        else:
            out_map[k] = result
    return out_map

# create a camera and get its property
cam = cv2.VideoCapture(0)
properties = poll_props(cam,prop_map)

# list our properties
for k,v in properties.items():
    print "{0:<12}\t:{1:<12}".format(k,v)

while(True):
    # toggle properties and get results. 
    sat = get_prop(cam,"saturation",prop_map)
    if( sat > 0.5 ):
        set_prop(cam,"saturation",prop_map,0.1)
    else:
        set_prop(cam,"saturation",prop_map,1.0)
    time.sleep(0.05)
    ret, frame = cam.read()
    cv2.imshow('Basic Web Cam',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# put our toys back on the shelf 
cam.release()
cv2.destroyAllWindows()

hue         	:0.5         
saturation  	:0.10000000149
format      	:None        
brightness  	:0.6171875   
pos_frame   	:None        
convert_rgb 	:None        
rectification	:None        
height      	:480.0       
contrast    	:0.315789461136
width       	:640.0       
pos_msec    	:None        
fps         	:None        
fourcc      	:None        
mode        	:None        
avi_ratio   	:None        
frame_count 	:None        
gain        	:None        
exposure    	:None        


KeyboardInterrupt: 

<h1 align=center> Go Pro Cameras</h1>
<img src="/files/images/gopro.jpg" align=center>
<h2>
<ul align=left>
<li>All GoPro Cameras through 3+ use a similar interface.</li>
<br>
<li>Each camera acts as a wifi access point with a simple web server.</li>
<br>
<li>We can hack this interface for nefarious purposes.</li>
<br>
<li>Default path is http://10.5.5.9:8080/</li>
<br>
<li><a href="http://goprohero.readthedocs.org/en/latest/">GoProHero</a> is a nice library wrapping a lot of commands.</li>
</h2>
</ul>



<h1 align=center>Some Specifics on GoPros</h1>
<img src="/files/images/gopro_screen.jpg" align=center>
<h2>
<ul align=left>
<li>Interface is hitting specific routes with parameters.</li>
<br>
<li>Images and video are collected using urllib and beautiful soup.</li>
<br>
<li>Naming conventions are a little convoluted -- need some helper code.</li>
<br>
<li>Control over resolutions and frame rates. Gain and exposure not so much.</li>
<br>
<li>Live streaming is a bit wonky.</li>
</h2>
</ul>

In [1]:
from goprohero import GoProHero # helper lib
import urllib # used to "wget" files
import bs4 # beautiful soup, to parse file
import time
from IPython.display import Image

def download_and_save(name,route="http://10.5.5.9:8080/videos/DCIM/100GOPRO/"):
    """
    Download a specific file on a connected GoPro camera.
    """
    grab = route+name
    result = urllib.urlopen(grab) # basically wget
    if( result.code == 200 ): # 200 means OK
        with open(name,'wb') as fp: 
            fp.write(result.read()) # write to file
    result.close()

def get_new(last=[],url="http://10.5.5.9:8080/videos/DCIM/100GOPRO/"):
    """
    Find files recently added to our go pro. Requires a list of files
    """
    unique = None
    last = set(last)
    out = urllib.urlopen(url)
    if(out.code == 200):
        soup = bs4.BeautifulSoup(out.read()) # read the file tree
        # make a set of all <a href> in the tree
        fresh = set([row.renderContents() for row in soup.findAll('a')]) 
        # do a set difference with our initial set
        unique = list(fresh.difference(last))
    return unique



In [5]:
# check the camera file system
out = get_new()
# setup our controller
cam = GoProHero(password="herpderp")
# put the camera in burst mode
cam.command('mode','burst')
# tell it to take a shot
cam.command('record','on')
# wait to finish
time.sleep(3)
# get the new files
out = get_new(last=out)
for o in out:
    print o
    download_and_save(o)

KeyboardInterrupt: 

In [ ]:
Image(out[0])
